# Sentiment Analysis
> * Positive or Negative

In [ ]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re

## Pre-Processing

In [2]:
from collections import namedtuple
from gensim.models import doc2vec
import multiprocessing
from konlpy.tag import Mecab
from konlpy.utils import pprint
import numpy as np
from ckonlpy.tag import Twitter as ctwitter
mecab = Mecab()
ct = ctwitter()

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
from sklearn.linear_model import LogisticRegression

In [ ]:
positiveSentence = pd.read_csv('./data/sentiment_data/positive_sentiment_sentence_from_news.csv', header=None)

In [ ]:
negativeSentence = pd.read_csv('./data/sentiment_data/negative_sentiment_sentence_from_news.csv', header=None)

In [ ]:
print (positiveSentence.shape)
positiveSentence.head()

In [ ]:
positiveSentence = pd.concat([positiveSentence, pd.DataFrame(len(positiveSentence)*[1], columns=[1])],axis = 1)

In [ ]:
positiveSentence.head()

In [ ]:
print (negativeSentence.shape)
negativeSentence.head()

In [ ]:
negativeSentence = pd.concat([negativeSentence, pd.DataFrame(len(negativeSentence)*[0], columns=[1])],axis = 1)
negativeSentence.head()

In [ ]:
newsSentence = pd.concat([positiveSentence, negativeSentence], axis = 0)
newsSentence.reset_index(drop=True, inplace = True)
print (newsSentence.shape)
newsSentence.head()

## 기자명 & 이메일 제거 및 50자 이내 제거

In [ ]:
newsSentence[0] = newsSentence[0].str.replace('^[가-힣]{3,4}[//. ]?기자[//. ]','')

In [ ]:
newsSentence[0] = newsSentence[0].str.replace('^[ ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]*','')

In [ ]:
newsSentence[0] = newsSentence[0].str.replace('[가-힣]{3,4}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+','')
print (newsSentence.shape)

In [ ]:
newsSentence.drop(newsSentence[newsSentence[0].str.len()<50].index, inplace = True)
newsSentence.reset_index(drop=True, inplace=True)
newsSentence.dropna(inplace=True)
newsSentence.reset_index(drop=True, inplace=True)
newsSentence.drop_duplicates(inplace = True)
newsSentence.reset_index(drop=True, inplace=True)
print (newsSentence.shape)

In [ ]:
newsSentence.to_csv('./data/sentiment_data/merged_sentiment_data.txt',index=None, header=None)

In [5]:
newsSentence = pd.read_csv('./data/sentiment_data/merged_sentiment_data.txt',encoding='utf-8', header=None)
newsSentence.head()

,0,1
0,주택조합 승인 자체가 몹시 어려웠던 시절 한국기자협회 주택조합 형식으로 추진된 이 ...,1
1,"서 의원은 여의도 당사에서 기자회견을 열어 “압도적으로 승리할 것이고 국민, 당원이...",1
2,박근혜 대통령이 14일 열린 새누리당 전당대회에 참석한 것은 그만큼 아직 여권에서 ...,1
3,"“살기 좋은 전북 만들기, 중앙정부와 연결고리 최선” “월드컵에 출전한 우리 국가대...",1
4,"""여러분 너무나 고맙고 감사합니다. 도와주신 여러분께 저의 마음속의 큰 절을 받아주...",1


## Doc2Vec

In [6]:
def read_data(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header
    return data

In [7]:
def tokenize(doc):
    return ['/'.join(t) for t in ct.pos(doc)]

In [ ]:
rating = read_data('./data/sentiment_data/ratings.txt')

In [ ]:
rating_doc = [(tokenize(row[1]), row[2]) for row in rating]
news_doc = [(tokenize(newsSentence.loc[idx][0]), newsSentence.loc[idx][1]) for idx in newsSentence.index]

In [ ]:
pickle.dump(rating_doc+news_doc, open('./data/pre_data/pre_data_for_sentiment_analysis.pickled','wb'))

In [ ]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

In [ ]:
tagged_run_docs = [TaggedDocument(d, [c]) for d, c in rating_doc+news_doc]
pickle.dump(tagged_run_docs, open('./data/pre_data/pre_data_tagged_run_docs.pickled','wb'))

In [8]:
from gensim.models.doc2vec import TaggedDocument

In [9]:
tagged_run_docs = pickle.load(open('./data/pre_data/pre_data_tagged_run_docs.pickled','rb'))

In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(tagged_run_docs, test_size=0.25, random_state=42)

In [13]:
doc_vectorizer = doc2vec.Doc2Vec(size=5000, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(train)

In [ ]:
for epoch in range(25):
    doc_vectorizer.train(train, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002  # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha  # fix the learning rate, no decay

In [ ]:
#To save
doc_vectorizer.save('./model/doc2vec_size5000_epoch25.model')
pprint(doc_vectorizer.most_similar('문재인/Noun'))
pprint(doc_vectorizer.most_similar('노무현/Noun'))
pprint(doc_vectorizer.most_similar('박근혜/Noun'))

In [ ]:
doc_vectorizer2 = doc2vec.Doc2Vec(size=15000, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer2.build_vocab(train)

In [ ]:
for epoch in range(25):
    doc_vectorizer2.train(train, total_examples=doc_vectorizer2.corpus_count, epochs=doc_vectorizer2.iter)
    doc_vectorizer2.alpha -= 0.002  # decrease the learning rate
    doc_vectorizer2.min_alpha = doc_vectorizer2.alpha  # fix the learning rate, no decay

In [ ]:
#To save
doc_vectorizer2.save('./model/doc2vec_size15000_epoch25.model')
pprint(doc_vectorizer2.most_similar('문재인/Noun'))
pprint(doc_vectorizer2.most_similar('노무현/Noun'))
pprint(doc_vectorizer2.most_similar('박근혜/Noun'))

In [ ]:
classifier = LogisticRegression(random_state=1234)